In [ ]:
from langchain import hub
from dotenv import load_dotenv
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults

load_dotenv()

search = TavilySearchResults()
# query = "what is the weather in SF"
# search.invoke(query)
tools = [search]

prompt = hub.pull("hwchase17/openai-functions-agent")
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "whats the weather in sf?"})

In [ ]:
from langchain.prompts import PromptTemplate 
from langchain.agents import initialize_agent                                   
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

search = TavilySearchResults()
tools = [search]

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# PREFIX = '''
# Bing Bong
# '''

# FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:
# '''
# Thought: Do I need to use a tool? Yes
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# '''

# When you have gathered all the information regarding AI algorithm, just write it to the user in the form of a blog post.

# '''
# Thought: Do I need to use a tool? No
# AI: [write a blog post]
# '''
# """

# SUFFIX = '''

# Begin!

# Previous conversation history:
# {chat_history}

# Instructions: {input}
# {agent_scratchpad}
# '''


agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=False,
    memory=memory,
    handle_parsing_errors=True,
    # return_intermediate_steps=True,
    # agent_kwargs={
        # 'prefix': PREFIX, 
    #     'format_instructions': FORMAT_INSTRUCTIONS,
    #     'suffix': SUFFIX
    # }
)

while True:
    query = input("Query: ")
    if query == "exit":
        break
    print("User: ", query)
    response = agent.invoke({"input": query})
    print("AI: ", response.get("output"))

In [ ]:
import inspect
print(inspect.getsource(initialize_agent))

In [ ]:
print(agent.agent.llm_chain.prompt.template)

In [ ]:
memory

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate

# Define your custom prompt template with required variables
prompt = hub.pull("hwchase17/structured-chat-agent")

# Initialize the Google Gemini model
llm = GoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# Set up Tavily as a search tool
tavily_tool = TavilySearchResults(max_results=10)

# Create the agent using your custom prompt template and Tavily tool
agent = create_react_agent(llm, tools=[tavily_tool], prompt=custom_prompt_template)

# Set up the agent executor
agent_executor = AgentExecutor(agent=agent, tools=[tavily_tool], verbose=True)

# Invoke the agent with a query
response = agent_executor.invoke({"input": "What is Python?"})
print(response)